In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


# Defining inputs and collecting them from Mnist

In [4]:
mnist = input_data.read_data_sets("tmp/data/",one_hot=True)
x_raw,y_raw = mnist.train.next_batch(20)
print(np.shape(x_raw))
print(y_raw)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz
(20, 784)
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [5]:
n_input = 28*28 #image_width*image_height
n_hidden = 2
n_classes = 10


tf.reset_default_graph()


# Defining weights and biases 

In [6]:
W = { "h1": tf.Variable(tf.ones([3,3,1,32])),
        "h2": tf.Variable(tf.ones([3,3,32,64])),
        "h3": tf.Variable(tf.ones([12*12*64,128])),
        "out": tf.Variable(tf.ones([128,n_classes])),
}

b = { "b1": tf.Variable(tf.zeros([32])),
        "b2": tf.Variable(tf.zeros([64])),
        "b3": tf.Variable(tf.zeros([128])),
        "bout": tf.Variable(tf.zeros([n_classes])),
}
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



In [7]:
#reshaping inputs

In [8]:
xi = tf.reshape(x,[-1,28,28,1])

# Biulding model

In [9]:
l1 = tf.nn.conv2d(xi,W["h1"],strides=[1,1,1,1],padding="VALID")
l1_b = tf.nn.bias_add(l1,b["b1"])
l1_act = tf.nn.relu(l1_b)

l2 = tf.nn.conv2d(l1_act,W["h2"],strides=[1,1,1,1],padding="VALID")
l2_b = tf.nn.bias_add(l2,b["b2"])
l2_act = tf.nn.relu(l2_b)

l3_max = tf.nn.max_pool(l2_act,[1,2,2,1],strides=[1,2,2,1],padding="VALID")

# Dense
l4 = tf.reshape(l3_max,[-1,12*12*64])
l4 = tf.matmul(l4,W["h3"])
l4_b = tf.nn.bias_add(l4,b["b3"])
l4_act = tf.nn.relu(l4_b)

l5 = tf.matmul(l4,W["out"])+b["bout"]
out_act = tf.nn.softmax(l5)


In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=l5,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Training model

In [11]:
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        x_raw,y_raw = mnist.train.next_batch(20)
        #pred = l1_b.eval({x: x_raw})
        #print(pred)
        for i in range(1):
              if(i%100==0):
                    print(i)
              kk = sess.run(optimizer,feed_dict={x: x_raw,y: y_raw})
        pred = out_act.eval({x: x_raw})
        
      

0


# Results 

In [12]:
np.shape(pred)

(20, 10)

In [13]:
for i in range(20):
    print("pred is :{} , actual value is: {}".format(np.argmax(pred[i]),np.argmax(y_raw[i])))
#bc of initializing ones it's take so many time to train good


pred is :3 , actual value is: 1
pred is :3 , actual value is: 6
pred is :3 , actual value is: 7
pred is :3 , actual value is: 1
pred is :3 , actual value is: 9
pred is :3 , actual value is: 7
pred is :3 , actual value is: 6
pred is :3 , actual value is: 5
pred is :3 , actual value is: 5
pred is :3 , actual value is: 8
pred is :3 , actual value is: 8
pred is :3 , actual value is: 3
pred is :3 , actual value is: 4
pred is :3 , actual value is: 4
pred is :3 , actual value is: 8
pred is :3 , actual value is: 7
pred is :3 , actual value is: 3
pred is :3 , actual value is: 6
pred is :3 , actual value is: 4
pred is :3 , actual value is: 6


# for evaluating some weights or etc just use:

In [ ]:
#evaluating weights of first hidden layer:
m = W["h1"].eval()

In [15]:

tf.trainable_variables()